## LDA Preprocessing 1
# Artifact Removal and Stopword Selection
This notebook takes the clean JSON files for each article and does some preprocessing to obtain a text that we can analyze using LDA.

Specifically, we do:
* Artifact removal
* Stopword selection

In the next notebook we will do:
* Punctuation removal
* Lemmatization

In [1]:
import json
import re

We will also use some utility functions we defined in the `utils/` folder:
* `loadCorpusList(path)`: Loads the corpus as a list of `Article` objects (see `utils/Article.py`). This will allow us to save the clean text per document into the same JSON file with the metadata included.
* `saveCorpus(path)`: Saves the articles in JSON format in their current state. Useful when we want to append information to our clean JSON files.

In [2]:
import os
import sys 

# Jupyter Notebooks are not good at handling relative imports.
# Best solution (not great practice) is to add the project's path
# to the module loading paths of sys.

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.loaders import loadCorpusList, saveCorpus

## Loading the corpus

In [3]:
corpusPath = '../data/parsedHTML'

corpusList = loadCorpusList(corpusPath)

In [4]:
corpusList[0].title

"La diferencia 'es/debe' y el argumento de la pregunta cerrada"

We will only work with documents in Spanish. Hence, let's replace `corpusList` with only the articles that are in Spanish.

In [5]:
corpusList = [doc for doc in corpusList if doc.lang == 'es']

This leaves is with 554 articles.

In [6]:
len(corpusList) 

991

## Artifact removal

There are some artifacts included in the text that are produced by HTML processing (or in the future because of how PDF files store text).

Let's start by removing numbers and some special characters such as newline characters (`\n`). We will keep normal punctuation for now as that might help SpaCy when we do lemmatization.

In [7]:
for doc in corpusList:
    doc.cleanText = re.sub('\d|\n',' ', doc.text)

We can detect some of these artifacts by looking for non-alphanumeric characters between alphanumeric characters (e.g. `"ar-gument"`, `"ar\xadgument"`).

In [8]:
artifacts = re.compile('\w+[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ\d\s:]\w+')

In [9]:
[re.findall(artifacts, doc.text) for doc in corpusList][0]

['ES/DEBE',
 'G.E',
 'auto-responde',
 'auto-respond',
 'no-natura',
 'no-naturale',
 'ES/DEBE',
 'es/debe',
 'es/deb',
 'no-valorativo',
 'afirmacic»ies',
 'ES/DEBE',
 'no-concluyente',
 'ai;giiment',
 'auto-respond',
 'no-concluyent',
 'auto-responde',
 'no-concluyentes',
 'hecho/valo',
 'ES/DEBE',
 'no-concluyente',
 'auto-respond',
 'no-concluyente',
 'es/deb',
 'no-concluyente',
 'G.E',
 'R.M']

One common artifact is the hex `\xad` for the soft hyphen which is used to break lines. We can remove it easily.

In [10]:
for doc in corpusList:
    doc.cleanText = re.sub('\\xad','', doc.cleanText)
    doc.cleanText = doc.cleanText.replace(u"\xa0", "")

We can save the corpus for now.

In [11]:
saveCorpus('../data/corpus', corpusList)

## Stopword Removal
Stopword removal is perhaps the most difficult part of preprocessing. There are two challenges to meet:
* Some stopword lists such as the one included in NLTK for Spanish are too weak and do not filter many stopwords.
* Other stopword lists are too inclusive and can eliminate words that are meaningful in philosophy (e.g. 'verdadero', true). 
It is important to note that stopwords are very context-sensitive. A word in one context may provide little meaning (hence counting as a stopword) while in other contexts it may provide lots of information.

To tackle these challenges, we will first to an initial filtering with NTLK's list. This will leave many stopwords in the text, but will reduce the size of each text considerably. Then we will compare the text with a stronger list of stopwords (source). We will see which words are both the text and the stronger stopwords list. We will inspect these lists manually and extract a list of protected words. We will iterate over this process a number of times. Once we have a robust list of protected words, we will concatenate NLTK's stopwords list with the stronger one and eliminate the protected words from it. This will provide a final (hopefully middle ground) stopword list with which to continue.

In [12]:
from nltk.corpus import stopwords as nltk_stopwords

stopwords_weak = nltk_stopwords.words('spanish')

In [13]:
import requests

r = requests.get('https://raw.githubusercontent.com/stopwords-iso/stopwords-es/master/stopwords-es.txt')
stopwords_strong = r.text.split('\n')

In [14]:
docWords = []
for doc in corpusList:
    docWords += [word for word in re.findall('\w+', doc.cleanText) if word not in stopwords_weak]

In [15]:
docWords[:10]

['L', 'A', 'DIFERENCI', 'A', 'ES', 'DEBE', 'Y', 'E', 'L', 'ARGUMENT']

In [16]:
from collections import Counter

docs_and_stopwords = Counter([word for word in docWords if word in stopwords_strong])

In [17]:
docs_and_stopwords.most_common(10)

[('s', 169541),
 ('l', 132270),
 ('n', 130995),
 ('d', 95875),
 ('r', 54215),
 ('u', 21476),
 ('ser', 17683),
 ('si', 12842),
 ('puede', 12369),
 ('i', 9685)]

Already in the first 10 most common words in both the documents and the strong list of stopwords we find words that in philosophy are quite meaningful:
* 'ser': being
* 'bien': good
* 'modo': mode

We will start saving those words and eliminating them from the stronger list of stopwords. Then we will repeat the process of selecting the words that are in both lists and see which words are common. By iterating over this process a couple of times, we will get a list of protected words.

In [18]:
protectedWords = [
    'ser',
    'bien',
    'modo'
]

In [19]:
stopwords_strong = [word for word in stopwords_strong if word not in protectedWords]
docs_and_stopwords = Counter([word for word in docWords if word in stopwords_strong])

In [20]:
docs_and_stopwords.most_common(100)

[('s', 169541),
 ('l', 132270),
 ('n', 130995),
 ('d', 95875),
 ('r', 54215),
 ('u', 21476),
 ('si', 12842),
 ('puede', 12369),
 ('i', 9685),
 ('mismo', 9186),
 ('sino', 9169),
 ('t', 7809),
 ('decir', 7218),
 ('parte', 6280),
 ('manera', 6272),
 ('dos', 5629),
 ('tal', 5395),
 ('hace', 5326),
 ('pues', 5056),
 ('así', 5047),
 ('misma', 4764),
 ('lugar', 4676),
 ('debe', 4663),
 ('embargo', 4656),
 ('posible', 4616),
 ('sólo', 4584),
 ('hecho', 4507),
 ('ejemplo', 4278),
 ('vez', 4246),
 ('p', 4190),
 ('entonces', 4164),
 ('verdad', 4115),
 ('tiempo', 4013),
 ('solo', 3889),
 ('ello', 3826),
 ('según', 3703),
 ('toda', 3665),
 ('m', 3497),
 ('h', 3475),
 ('cosas', 3458),
 ('poder', 3453),
 ('parece', 3306),
 ('b', 3304),
 ('cada', 3254),
 ('respecto', 3182),
 ('pueden', 3180),
 ('saber', 3165),
 ('aquí', 3152),
 ('da', 3120),
 ('f', 3110),
 ('hacer', 3096),
 ('primer', 3010),
 ('general', 3000),
 ('primera', 2963),
 ('partir', 2963),
 ('menos', 2942),
 ('cómo', 2906),
 ('trata', 2869),

In [21]:
protectedWords += [
    'parte',
    'posible',
    'lugar',
    'hecho',
    'poder',
    'verdad',
    'cosas',
    'general',
    'fin',
    'trabajo',
    'cierto',
    'uso',
    'dado',
    'diferentes',
    'verdadero',
    'verdadera',
    'existe',
    'valor',
    'realizar',
    'existen',
    'conocer',
    'diferente'
]

protectedWords = list(set(protectedWords))

After a couple of times to make the process less complex, once we are sure of a set of words, we can eliminate those from the list of document words and go back and repeat the process a couple more times.

In [22]:
stopwordsToRemove = [word[0] for word in docs_and_stopwords.most_common(100)]
docWords = [word for word in docWords if word not in stopwordsToRemove]

### Removing stopwords in English

Given that most of the articles have abstracts in English, some of the usual stopwords in English are appearing frequently in our documents. Thus, we will append the NLTK-generated list of English stopwords. We will also use one for Portuguese, which we sometimes get as well.

In [23]:
englishStopwords = nltk_stopwords.words("english")
portugueseStopwords = nltk_stopwords.words("portuguese")

TODO: should we be filtering these just like we filter the Spanish ones?

### Other stopwords custom to our corpus

There are some other stopwords that we would like to include, but that have not been taken into account in the previous processes. These are:

In [24]:
custom_stopwords = [
    "cf",
    "no obstante",
    "sin embargo",
    "por ejemplo",
    "es decir",
    "ak",
    "krv"
]

Once we have a robust set of words we can save both the final stopword list and the protected words list.

In [25]:
stopwords_final = list(set(
    stopwords_weak + stopwords_strong + englishStopwords + portugueseStopwords +  custom_stopwords
))
with open('wordlists/stopwords.txt', 'w') as fp:
    fp.write('\n'.join(stopwords_final))

with open('wordlists/protectedWords.txt', 'w') as fp:
    fp.write('\n'.join(protectedWords))